## Zero Shot Inference with an Instruction 

Zero-shot learning in NLP allows a pre-trained LLM to generate responses to tasks that it hasn’t been specifically trained for. In this technique, the model is provided with an input text and a prompt that describes the expected output from the model in natural language. The pre-trained models can use its knowledge to generate coherent and relevant responses even for prompts it hasn’t specifically been trained on. Zero-shot learning can reduce the time and data required while improving efficiency and accuracy of NLP tasks. Zero-shot learning is used in a variety of NLP tasks, such as question answering, summarization, and text generation.

We were asked to explore the use of LLMs in prediction tasks. Using as templates for the questions to the model the following website: https://polymarket.com/. 

## Polymarket API
We found an API but it seems to be design to complete orders
https://docs.polymarket.com/#system

***The Polymarket Order Book API is a set of endpoints that allow market makers, traders, and other Polymarket users to programmatically create and manage orders for markets via access to the API provided by the operator.***

We do not want to perform any trading but to collect questions from the markets so this is not our place...

## Target models
From Ollama (https://ollama.ai/) we use models llama2 and mistral.

### Pretraining data cutoff?
* For Llama2 the pretraining data has a cutoff of September 2022
https://llama-2.ai/llama-2-model-details/
* Mistral 7B’s training data cutoff was sometime in 2023, but we could not find an exact date. However it was released around October 2023.

## How to build the training dataset?
Collecting questions from closed markets and from different years. However in the polymarket website most of the closed questions are from 2023. Thus we have more testing data for llama2 rather than mistral.
We copied manually just some dozens of questions due to lack of time. However this volume is not big enough to achieve any relevant conclusions so we will use it just to research a bit the "prediction skills" of LLM´s.

### Warning about "prediction" in LLMs

<div style="padding: 15px; border: 1px solid transparent; border-color: transparent; margin-bottom: 20px; border-radius: 4px; color: #8a6d3b;; background-color: #fcf8e3; border-color: #faebcc;">
The very same creators of these LLMs warn us that the reliability of these models is not 100%. An LLM can create different answers not based on any reference, and this behaviour remains even though reducing the temperature parameter of the model to the minimum. Having said that the scope of this exercise is maybe not well aligned with the true non-reliable-nature of the LLMs.   
</div>

<div align="center">
    <img width="800" src="./mistral_lying.png" alt="mistral_lying" />

</div>

## Selecting a business
We checked different businesses and the volumen of questions for closed markets.

* Crypto
* Politics
* Sports
* Science

The Sports category seems to have a big amount of questions and maybe there is some reference information in the past from those teams, people that could help in the prediction. Besides the Science category has questions that could be asked out of time. Thus we select some questions of these two categories.

## Preparing questions and answers
We wrote files with questions and answers for both categories in separate files contained within the data folder.

In [8]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
cwd = os.getcwd()
parent_dir = os.path.abspath(os.path.join(cwd, os.pardir))
data_path = os.path.join(parent_dir, "data")
with open(os.path.join(data_path,'sport_questions.txt'),encoding='utf-8') as f:
    sport_questions = f.read().splitlines() 
    f.close()


In [24]:
sport_questions

['In the cricket IPL game, scheduled for May 14th in 2023, which team is going to win: Rajasthan or Bangalore?',
 'In the NBA basketball game, scheduled for October 27th in 2023, which team is going to win: Pistons or Hornets?',
 'Will Netherlands qualify for the World Cup in 2023?',
 'Will Senegal qualify for the World Cup in 2023?',
 'Will France qualify for the World Cup in 2023?',
 'Will Spain qualify for the World Cup in 2023?',
 'Will Qatar qualify for the World Cup in 2023?',
 'Will Bryce Young be first QB selected in the 2023 NFL draft?',
 'Will Tadej Pogačar win the 2023 Tour de France?',
 'In the 2023 Qatar Open Final in Tennis, who is going to win Swiatek or Pegula',
 "Will Coco Gauff win Women's US Open in Tennis?",
 'On December the 19th of 2022 who is going to win the NFL Rams or Packers',
 'In the cricket IPL game, scheduled for April the 1st in 2023, which team is going to win: Punjab or Kolkata',
 'Will Hans Niemann finish top 3 in the U.S. Chess Championship in 2022?'

In [3]:
with open(os.path.join(data_path,'sport_answers.txt'),encoding='utf-8') as f:
    sport_answers = f.read().splitlines()
    f.close()

In [4]:
with open(os.path.join(data_path,'science_questions.txt'),encoding='utf-8') as f:
    science_questions = f.read().splitlines()
    f.close()

In [5]:
with open(os.path.join(data_path,'science_answers.txt'),encoding='utf-8') as f:
    science_answers = f.read().splitlines()
    f.close()

In [6]:
questions_df = pd.DataFrame(columns=["question", "answer", "category", "year"])

In [9]:
for i in range(len(sport_questions)):
    question = sport_questions[i]
    result = re.findall('(\d{4})', question)
    if len(result) > 0:
        year = int(result[0])
    else:
        year = 2024
    questions_df = questions_df.append({'question': question, 
                         'answer': sport_answers[i],
                         'category': "sport",
                         'year': year}, 
                        ignore_index=True)
    
questions_df.head()

,question,answer,category,year
0,"In the cricket IPL game, scheduled for May 14t...",Bangalore,sport,2023
1,"In the NBA basketball game, scheduled for Octo...",Pistons,sport,2023
2,"In the cricket IPL game, scheduled for May 14t...",Bangalore,sport,2023
3,"In the NBA basketball game, scheduled for Octo...",Pistons,sport,2023
4,Will Netherlands qualify for the World Cup in ...,Yes,sport,2023


In [10]:
for i in range(len(science_questions)):
    question = science_questions[i]
    result = re.findall('(\d{4})', question)
    if len(result) > 0:
        year = int(result[0])
    else:
        year = 2024
    questions_df = questions_df.append({'question': question, 
                         'answer': science_answers[i],
                         'category': "science",
                         'year': year}, 
                        ignore_index=True)
    
questions_df.tail()

,question,answer,category,year
29,Alien contact in 2023?,No,science,2023
30,Will SpaceX's Starship lift 500m from pad by N...,Yes,science,2023
31,Is the Room-Temperature Superconductor discove...,No,science,2023
32,Iceland volcanic eruption by November 17th 2023?,No,science,2023
33,Is the Earth flat?,No,science,2024


In [11]:
questions_df.year.value_counts()

2023    29
2022     3
2024     2
Name: year, dtype: int64

In [12]:
questions_df.to_pickle(os.path.join(data_path,'questions_df'))

## References
Mistral paper
https://arxiv.org/abs/2310.06825

Zero Shot in LLMs
https://arxiv.org/pdf/2205.11916.pdf